### Text Clean And Optimization 

This notebook used for text pre clean and optimization that includes specific language text clean techniques.  

#### Multi Processing Text Preprocess

In [2]:
import unicodedata
import numpy as np
import pandas as pd
import re
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice

In [3]:
#help(multiprocessing)

In [4]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [21]:
# custom character alphabet 
tr = re.compile(r"[abcçdefgğhıijklmnoöprsştuüvyzqxw0123456789\.\?\{\}\[\]\(\),:;!'\"\$`]+", re.IGNORECASE|re.UNICODE) # Turkish filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n0123456789'
en = re.compile(r"[abcdefghıijklmnopqrstxuvwyz0123456789\.\?\{\}\[\]\(\),:;!'\"\$`]+", re.IGNORECASE|re.UNICODE) # English
nl = re.compile(r"[abcdefghıijklmnopqrstxuvwyzāăēĕīĭōŏūŭ0123456789\.\?\{\}\[\]\(\),:;!'\"\$`]+", re.IGNORECASE|re.UNICODE) # Dutch (Flemenk)
fr = re.compile(r"[abcçdefghıijklmnopqrstxuvwyzàâæèéêëîïôœùûüÿ0123456789\.\?\{\}\[\]\(\),:;!'\"\$`]+", re.IGNORECASE|re.UNICODE) # French
de = re.compile(r"[abcdefghıijklmnopqrstxuvwyzäöüß0123456789\.\?\{\}\[\]\(\),:;!'\"\$`]+", re.IGNORECASE|re.UNICODE) # Deutch
es = re.compile(r"[abcdefghıijklmnopqrstxuvwyzñáéíóú0123456789\.\?\{\}\[\]\(\),:;!'\"\$`]+", re.IGNORECASE|re.UNICODE) # Spanish (¿¡)
pt = re.compile(r"[abcçdefghıijklmnopqrstxuvwyzàáâãéêíóôõú0123456789\.\?\{\}\[\]\(\),:;!'\"\$`]+", re.IGNORECASE|re.UNICODE) # Portuguese
it = re.compile(r"[abcdefghıijklmnopqrstxuvwyzàéèìòùî0123456789\.\?\{\}\[\]\(\),:;!'\"\$`]+", re.IGNORECASE|re.UNICODE) # Italian
ar = re.compile(r"[ٿصؼۤڳڲؿڎػڠجڿ٬ٸؽؒؓطۄڀۂؘؔتٚڛےٝڜؖڦ٫ډ۰زۇٖۀ،لۓ۱عٮێڔ۶ؚۧۜڤۏ٤إٞٷؗۖ؈ژۣؕؑٴأۻڸۺگاڴڹۯ؉ْڌ٣؍ي٧؟ـٟړۅؐڶُىڽېًۢؠضۚڄٛڏٱۦ٩س٦ڼڂٔۘ٠ښٌٍ۬ٳ۾ٲږذۋٵٜ٘ڞڅںٗهڣۿپڒۥۗڋیؙم؞ثۨ۹ٹڵڪظٶۭ١۔ڭەڨحٕ؎ٺڷٰ۪۫۲ڻڥۛڑڟټآڡغګ؊ّ٢ٯڧڮ؏ۮ؋ؤ٪ؾڗۼق۟دکوِڰڐۃ۽ہفر۷ڇ٭چڝ۴بۈٽڕۡھ۸٨ةٓڃئ؛ڬٙڙڢڱۊَۆۉځ۠ۍۑۙڊنءڈٻش٥ڍ؇۵ك۳خ\.\?\{\}\[\]\(\)\"\$\ا0123456789,':;\"`]+", re.IGNORECASE|re.UNICODE) # Arabic

In [16]:
#test = '''ٿصؼۤڳڲؿڎػڠجڿ٬ٸؽؒؓطۄڀۂؘؔتٚڛےٝڜؖڦ٫ډ۰زۇٖۀ،لۓ۱عٮێڔ۶ؚۧۜڤۏ٤إٞٷؗۖ؈ژۣؕؑٴأۻڸۺگاڴڹۯ؉ْڌ٣؍ي٧؟ـٟړۅؐڶُىڽېًۢؠضۚڄٛڏٱۦ٩س٦ڼڂٔۘ٠ښٌٍ۬ٳ۾ٲږذۋٵٜ٘ڞڅںٗهڣۿپڒۥۗڋیؙم؞ثۨ۹ٹڵڪظٶۭ١۔ڭەڨحٕ؎ٺڷٰ۪۫۲ڻڥۛڑڟټآڡغګ؊ّ٢ٯڧڮ؏ۮ؋ؤ٪ؾڗۼق۟دکوِڰڐۃ۽ہفر۷ڇ٭چڝ۴بۈٽڕۡھ۸٨ةٓڃئ؛ڬٙڙڢڱۊَۆۉځ۠ۍۑۙڊنءڈٻش٥ڍ؇۵ك۳خ\.\?\{\}\[\]\(\)\"\$\ا0123456789,':;\"`'''

In [ ]:
def convert_one_character_letter(text):
    '''
    This function converts two byte occupy of letter to one byte unicode character without any visual change \n
    like as Turkish character ç,ş,ö,ğ. \n
    convert_one_character_letter(text): text is any string word or sentence.
    '''
    new_text = unicodedata.normalize('NFC', f"{text}")
    return new_text

In [7]:
def custom_character(text_data, custom_alp):  # custom_character(text_data, tr)
    '''
    custom_character(text_data, custom_alp) text_data is any string word,sentence or plain text data.\n
    function clean string according to specific language alphabet\n
    '''
    text_one = convert_one_character_letter(text_data)
    text_data2 = re.findall(custom_alp, text_one)
    text_data3 = " ".join(text_data2)  # with one whitespace
    
    return text_data3  

In [ ]:
# this technique uses in Rev1 notebook
#def custom_character(text_data, custom_alp):  # custom_character(text_data, tr)
#    '''
#    custom_character(text_data, custom_alp) text_data is any string word,sentence or plain text data.\n
#    function clean string according to specific language alphabet\n
#    '''
#    text_data2 = re.findall(custom_alp, text_data)
#    text_data3 = " ".join(text_data2)  # with one whitespace
#    
#    return text_data3 

In [ ]:
def english_edit(text):  # english_edit(text)
    '''
    english_edit(text) use for plain english text optimization because of not affecting text cleaning process.
    '''
    not_case_sens1 = re.compile(re.escape("can\'t"), re.IGNORECASE)
    text = not_case_sens1.sub("can not", text)
    
    not_case_sens2 = re.compile(re.escape("won\'t"), re.IGNORECASE)
    text2 = not_case_sens2.sub("will not", text)
    
    not_case_sens3 = re.compile(re.escape("n\'t"), re.IGNORECASE)
    text3 = not_case_sens3.sub(" not", text2)
    
    not_case_sens4 = re.compile(re.escape("\'ll"), re.IGNORECASE)
    text4 = not_case_sens4.sub(" will", text3)
    
    not_case_sens5 = re.compile(re.escape("\'m"), re.IGNORECASE)
    text5 = not_case_sens5.sub(" am", text4)
    
    not_case_sens6 = re.compile(re.escape("\'ve"), re.IGNORECASE)  # 'd, 's, 're
    text6 = not_case_sens6.sub(" have", text5)
    
    not_case_sens7 = re.compile(re.escape("\'"), re.IGNORECASE)
    text7 = not_case_sens7.sub(" \'", text6)
    
    return text7

In [ ]:
folder_name = "Turkish"  # English, Turkish, German, French, Spanish, Arabic, Portuguese, Dutch, Italian
 
if folder_name == "English":
    custom_alp = en
    text_name = "en"
elif folder_name == "Turkish":
    custom_alp = tr
    text_name = "tr"
elif folder_name == "German":
    custom_alp = de
    text_name = "de"
elif folder_name == "French":
    custom_alp = fr
    text_name = "fr"
elif folder_name == "Spanish":
    custom_alp = es
    text_name = "es"
elif folder_name == "Arabic":
    custom_alp = ar
    text_name = "ar"
elif folder_name == "Portuguese":
    custom_alp = pt
    text_name = "pt"
elif folder_name == "Dutch":
    custom_alp = nl
    text_name = "nl"
elif folder_name == "Italian":
    custom_alp = it
    text_name = "it"
else:
    pass
#custom_alp = tr  # en, tr, de, fr, es, ar, pt, nl, it
print(f"{folder_name}\n{text_name}\n{custom_alp}") 

In [ ]:
plain_text_path = "/home/ubuntu/Workspace"  # tr1.txt, tr2.txt vb. Note: divided plain text part size should be less than 1GB for 64GB RAM machine

In [ ]:
def part_1():  
  part1_num = 1
  try:
    with open(f"{plain_text_path}/{text_name}{part1_num}.txt", "r", encoding="utf-8", errors="ignore") as file:
      text_data = file.read()
  except:
    print("There is not such a file  or path is incorrect")
    
  if text_name == "en":
    text = english_edit(text_data)  # english edit text
    text_result = custom_character(text, custom_alp)  # custom character alphabet
  else:
    apostro = re.compile(re.escape("\'"), re.IGNORECASE)
    text = apostro.sub(" \'", text_data)
    text_result = custom_character(text, custom_alp)  # custom character alphabet

  with open(f"{text_name}{part1_num}.txt", "w", encoding="utf-8") as file:  # output save in notebook folder
    file.write(text_result)  
  

In [ ]:
def part_2():  
  part2_num = 2
  try:
    with open(f"{plain_text_path}/{text_name}{part2_num}.txt", "r", encoding="utf-8", errors="ignore") as file:
      text_data = file.read()
  except:
    print("There is not such a file  or path is incorrect")
  
  if text_name == "en":
    text = english_edit(text_data)  # english edit text
    text_result = custom_character(text, custom_alp)  # custom character alphabet
  else:
    apostro = re.compile(re.escape("\'"), re.IGNORECASE)
    text = apostro.sub(" \'", text_data)
    text_result = custom_character(text, custom_alp)  # custom character alphabet

  with open(f"{text_name}{part2_num}.txt", "w", encoding="utf-8") as file:  # output save in notebook folder
    file.write(text_result)

In [ ]:
def part_3():  
  part3_num = 3
  try:
    with open(f"{plain_text_path}/{text_name}{part3_num}.txt", "r", encoding="utf-8", errors="ignore") as file:
      text_data = file.read()
  except:
    print("There is not such a file  or path is incorrect")
  
  if text_name == "en":
    text = english_edit(text_data)  # english edit text
    text_result = custom_character(text, custom_alp)  # custom character alphabet
  else:
    apostro = re.compile(re.escape("\'"), re.IGNORECASE)
    text = apostro.sub(" \'", text_data)
    text_result = custom_character(text, custom_alp)  # custom character alphabet

  with open(f"{text_name}{part3_num}.txt", "w", encoding="utf-8") as file:  # output save in notebook folder
    file.write(text_result)

In [ ]:
def part_4():  
  part4_num = 4
  try:
    with open(f"{plain_text_path}/{text_name}{part4_num}.txt", "r", encoding="utf-8", errors="ignore") as file:
      text_data = file.read()
  except:
    print("There is not such a file  or path is incorrect")
  
  if text_name == "en":
    text = english_edit(text_data)  # english edit text
    text_result = custom_character(text, custom_alp)  # custom character alphabet
  else:
    apostro = re.compile(re.escape("\'"), re.IGNORECASE)
    text = apostro.sub(" \'", text_data)
    text_result = custom_character(text, custom_alp)  # custom character alphabet

  with open(f"{text_name}{part4_num}.txt", "w", encoding="utf-8") as file:  # output save in notebook folder
    file.write(text_result)

In [ ]:
def part_5():  
  part5_num = 5
  try:
    with open(f"{plain_text_path}/{text_name}{part5_num}.txt", "r", encoding="utf-8", errors="ignore") as file:
      text_data = file.read()
  except:
    print("There is not such a file  or path is incorrect")
  
  if text_name == "en":
    text = english_edit(text_data)  # english edit text
    text_result = custom_character(text, custom_alp)  # custom character alphabet
  else:
    apostro = re.compile(re.escape("\'"), re.IGNORECASE)
    text = apostro.sub(" \'", text_data)
    text_result = custom_character(text, custom_alp)  # custom character alphabet
  
  with open(f"{text_name}{part5_num}.txt", "w", encoding="utf-8") as file:  # output save in notebook folder
    file.write(text_result)

In [ ]:
# multi process code block
if __name__ == '__main__':
    p1 = Process(target=part_1,)
    p2 = Process(target=part_2,)
    p3 = Process(target=part_3,)
    p4 = Process(target=part_4,)
    p5 = Process(target=part_5,)
    
    p1.start()
    p2.start()
    p3.start()
    p4.start()
    p5.start()
    
    p1.join()
    p2.join()
    p3.join()
    p4.join()
    p5.join()